Tải các thư viện

In [1]:
import cv2
import numpy as np

Khởi tạo đọc video và gọi biến bằng phương pháp MOG2

In [2]:
# Tạo đối tượng trừ nền (sử dụng MOG2) để tách nền khỏi đối tượng chuyển động
backSub = cv2.createBackgroundSubtractorMOG2()

# Đọc video đầu vào từ đường dẫn
cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/video/video_mog2.mp4')

Thiết lập thông số cho việc ghi video

In [3]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Định dạng mã video codec cho file mp4
frame_width = int(cap.get(3))  # Độ rộng của khung hình video
frame_height = int(cap.get(4))  # Chiều cao của khung hình video

Khởi tạo đầu output của video

In [4]:
# Khởi tạo đối tượng ghi video chính thức sau khi xử lý
out = cv2.VideoWriter('output2.mp4', fourcc, 20.0, (frame_width, frame_height))

# Khởi tạo đối tượng ghi video lưu ảnh sau khi trừ nền nhưng chưa khử nhiễu
out_no_denoise = cv2.VideoWriter('output_no_denoise.mp4', fourcc, 20.0, (frame_width, frame_height))

# Khởi tạo đối tượng ghi video lưu ảnh sau khi trừ nền và đã khử nhiễu
out_noise = cv2.VideoWriter('output_noise.mp4', fourcc, 20.0, (frame_width, frame_height))

Đặt tọa độ cho khung hình chữ nhật với mục đích phát hiện đối tượng chuyển động trong khung hình đó

In [5]:
# Đặt tọa độ cho hình chữ nhật ở giữa khung hình
center_x = frame_width // 2  # Tọa độ x của trung tâm khung hình
center_y = frame_height // 2  # Tọa độ y của trung tâm khung hình
top_left = (center_x - frame_width // 4 #trừ đi một phần tư độ rộng (frame_width // 4) để dịch chuyển sang bên trái.
            , center_y - frame_height // 4 )#trừ đi một phần tư chiều cao (frame_height // 4) để dịch chuyển lên trên.
bottom_right = (center_x + frame_width // 4, center_y + frame_height // 4)  # Góc dưới bên phải của hình chữ nhật

Biến đếm tổng số lượng đối tượng phát hiện trong khung hình

In [6]:
total_detected_objects = 0
total_true_positives = 0
total_false_positives = 0
total_false_negatives = 0

Khởi tạo vòng lặp xử lí các khung hình của video

In [7]:

while True:
    ret, frame = cap.read()  # Đọc từng khung hình của video, trả về 'ret' (True nếu đọc được, False nếu không) và 'frame' là khung hình
    if not ret:
        break  # Nếu không còn khung hình nào (ret=False), dừng vòng lặp

    # Áp dụng bộ trừ nền để tạo mặt nạ nền cho khung hình hiện tại
    fgMask = backSub.apply(frame)
    fgMask_color = cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)  # Chuyển ảnh mặt nạ sang ảnh màu (định dạng BGR) để có thể ghi vào video

    # Ghi khung hình sau khi trừ nền nhưng chưa khử nhiễu
    out_no_denoise.write(fgMask_color)

    # Khử nhiễu trên mặt nạ bằng các phép toán xử lý ảnh
    kernel = np.ones((5, 5), np.uint8)  # Tạo kernel 5x5 với tất cả phần tử là 1, loại uint8 (để thực hiện các phép toán giãn và co)
    fgMask = cv2.erode(fgMask, kernel, iterations=1)  # Phép co ảnh để loại bỏ các đốm nhiễu nhỏ; 1 lần lặp lại (iterations=1)
    fgMask = cv2.dilate(fgMask, kernel, iterations=1)  # Phép giãn ảnh để phục hồi vùng bị mất do phép co; 1 lần lặp lại (iterations=1)
    fgMask = cv2.GaussianBlur(fgMask, (3, 3), 0)  # Làm mượt bằng bộ lọc Gaussian với kích thước 3x3 và độ lệch chuẩn 0
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # Đóng (close) các lỗ nhỏ trong mặt nạ, sử dụng kernel 5x5
    _, fgMask = cv2.threshold(fgMask, 130, 255, cv2.THRESH_BINARY)  # Ngưỡng hóa để chuyển thành ảnh nhị phân: giá trị > 130 thành 255 (trắng), còn lại thành 0 (đen)

    # Chuyển đổi mặt nạ đã khử nhiễu thành ảnh màu để ghi vào video
    fgMask_color_denoised = cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)

    # Ghi lại khung hình sau khi trừ nền và khử nhiễu
    out_noise.write(fgMask_color_denoised)

    # Tìm đường viền của các đối tượng trong mặt nạ
    fsMask = cv2.Canny(fgMask, 20, 200)  # Sử dụng bộ lọc Canny để phát hiện cạnh với ngưỡng dưới 20 và ngưỡng trên 200
    contours, _ = cv2.findContours(fsMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # Tìm đường viền, sử dụng phương pháp RETR_TREE để lấy tất cả các đường viền và CHAIN_APPROX_SIMPLE để giảm số điểm

    # Vẽ hình chữ nhật phát hiện đối tượng trên khung hình gốc
    cv2.rectangle(frame, top_left, bottom_right, (255, 255, 0), 2)  # Vẽ hình chữ nhật màu xanh dương nhạt (BGR: 255, 255, 0) với độ dày viền 2 pixel

    # Đếm số lượng đối tượng trong khung hình hiện tại
    detected_objects = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for i in range(len(contours)):  # Duyệt qua từng đường viền được phát hiện
        (x, y, w, h) = cv2.boundingRect(contours[i])  # Tính tọa độ góc trên trái (x, y) và kích thước (w: width, h: height) của hình chữ nhật bao quanh đối tượng
        cx = x + w // 2  # Tọa độ x của tâm hình chữ nhật (vị trí x cộng nửa chiều rộng w)
        cy = y + h // 2  # Tọa độ y của tâm hình chữ nhật (vị trí y cộng nửa chiều cao h)

        # Kiểm tra xem đối tượng có nằm trong vùng hình chữ nhật phát hiện không
        logic = top_left[0] < cx < bottom_right[0] and top_left[1] < cy < bottom_right[1]  # Điều kiện: tâm của đối tượng phải nằm trong tọa độ vùng phát hiện
        area = cv2.contourArea(contours[i])  # Diện tích của đường viền

        if area < 500:  # Bỏ qua các đối tượng có diện tích nhỏ hơn 500 pixel (được coi là nhiễu hoặc không đủ lớn để tính là một đối tượng)
            continue
        if logic:
            true_positives += 1
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Vẽ hình chữ nhật màu xanh lá (BGR: 0, 255, 0) với độ dày viền 2 pixel quanh đối tượng phát hiện
            cv2.putText(frame, 'People', (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)  # Thêm nhãn 'People' tại vị trí (500, 100) trên khung hình, màu đỏ, cỡ chữ 2

            detected_objects += 1  # Tăng biến đếm đối tượng phát hiện trong khung hình
            total_true_positives += 1
        else:
            false_positives += 1

   # Cập nhật tổng số true positives, false positives, false negatives
    total_true_positives += true_positives
    total_false_positives += false_positives
    total_false_negatives += len(contours) - true_positives - false_positives

    total_detected_objects += detected_objects
    #Ghi lại video đã hoàn tất
    out.write(frame)


Hiển thị thông số và giải phóng tài nguyên

In [8]:

# Sau khi xử lý video xong, tính các thang đo đánh giá
# Tính Precision: Tỷ lệ dự đoán đúng trong số tất cả các dự đoán là dương tính
# Precision = true_positives / (true_positives + false_positives)
# Nếu mẫu số bằng 0 (trường hợp không có dự đoán dương tính nào), Precision được gán giá trị 0 để tránh lỗi chia cho 0
precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0

# Tính Recall: Tỷ lệ dự đoán đúng trong số tất cả các trường hợp thực sự dương tính
# Recall = true_positives / (true_positives + false_negatives)
# Nếu mẫu số bằng 0 (không có trường hợp dương tính nào), Recall được gán giá trị 0 để tránh lỗi chia cho 0
recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0

# Tính F1 Score: Trung bình điều hòa của Precision và Recall
# F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
# Nếu mẫu số bằng 0 (Precision + Recall = 0), F1 Score được gán giá trị 0 để tránh lỗi chia cho 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0


# In ra các thang đo đánh giá
print("Tổng số đối tượng được phát hiện:", total_detected_objects)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


# Giải phóng tài nguyên
cap.release()
out.release()
out_no_denoise.release()
out_noise.release()

Tổng số đối tượng được phát hiện: 10119
Precision: 0.7300075749377773
Recall: 0.12545873834556637
F1 Score: 0.21411907847753062
